LangChain Annual Report Analysis

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def annual_report(id,y):
  url = 'https://doc.twse.com.tw/server-java/t57sb01'
  #Input for POST REQUEST
  data = {
      "id":"",
      "key":"",
      "step":"1",
      "co_id":id,
      "year":y,
      "seamon":"",
      "mtype":'F',
      "dtype":'F04'
  }
  try:
    response = requests.post(url, data=data)
    #Extract file name
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a').text
    print(link1)
  except Exception as e:
    print(f"發生{e}錯誤")
  #2nd POST request
  data2 = {
      'step':'9',
      'kind':'F',
      'co_id':id,
      'filename':link1
  }
  try:
    response = requests.post(url, data=data2)
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a')
    #Get link for PDF
    link2 = link1.get('href')
    print(link2)
  except Exception as e:
    print(f"發生{e}錯誤")
  try:
    response = requests.get('https://doc.twse.com.tw' + link2)
    
    with open(y + '_' + id + '.pdf', 'wb') as file:
        file.write(response.content)
    print('OK')
  except Exception as e:
    print(f"發生{e}錯誤")

In [3]:
annual_report('2330','112')

2022_2330_20230606F04.pdf
/pdf/2022_2330_20230606F04_20240303_123547.pdf
OK


In [ ]:
!pip install langchain openai tiktoken pdfplumber==0.10.2 faiss-cpu

In [ ]:
import os
import getpass
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
llm_16k = ChatOpenAI(model="gpt-3.5-turbo-1106")

OpenAI API Key:··········


In [ ]:
#Function for building vector database
def pdf_loader(file,size,overlap):
  loader = PDFPlumberLoader(file)
  doc = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(
                          chunk_size=size,
                          chunk_overlap=overlap)
  new_doc = text_splitter.split_documents(doc)
  db = FAISS.from_documents(new_doc, OpenAIEmbeddings())
  return db

In [ ]:
db = pdf_loader('/content/112_2330.pdf',500,50)

In [ ]:
query = "公司是否有明確的成長或創新策略?"
docs = db.similarity_search(query,k=3)
for i in docs:
    print(i.page_content)
    print('_________')

5.4.1 客戶
提升同仁問題解決與創新的能力，以維持台積公司競爭
優勢並達到客戶滿意的雙贏目標。除了公司內部跨組織 台積公司的客戶遍布全球，產品種類眾多，在半導 最近二年度佔全年度合併營業收入淨額10%以上之客戶資料
的學習交流外，台積公司亦透過「台灣持續改善活動競 體產業的各個領域中表現傑出。客戶包括有無晶
單位：新台幣仟元
賽」，跨產業分享改善手法，期望能以台積公司的經驗分 圓廠設計公司、系統公司和整合元件製造商，例 民國111年 民國110年
名稱
享，促進其他在地產業的發展與進步，並藉由觀摩其他 如：Advanced Micro Devices, Inc.、Amazon Web 佔全年度營業收入 佔全年度營業收入
金額 與發行人之關係 金額 與發行人之關係
淨額比率（%） 淨額比率（%）
產業的改善方法，提升同仁問題解決與創新能力。民國 Services, Inc.、Broadcom Inc.、Intel Corporation、聯
甲客戶 529,649,200 23% 無 405,402,955 26% 無
_________
持其未來發展優勢。
我們將持續致力於優化製造營運（包括「數位化」我們的晶圓廠）來提高效率和生產力，藉以支援民國一百一十
二年與此後的N3高度量產。
我們正在增加台灣以外的產能以擴大我們的未來成長潛力、接觸全球人才，並進一步提升客戶信任。隨著我們擴 劉德音 魏魏哲哲家家
大全球足跡並在世界各地進行招募，我們的首要任務是識別、吸引和雇用與台積公司核心價值和原則相符的人 董事長 總裁
才，讓我們無論在何處營運，都能夠於所有員工間建立台積公司文化。
010 011
_________
能對台積公司的營運與財務表現產生負面影響。 台積公司的營運成長及產能的持續擴充計劃，仰賴來自 的營運成本，如果這些專利主張實體成功阻攔本公司所
若台積公司無法克服上述挑戰，則可能對公司業務、財 市場中少數的供應商以得到充分的設備及相關服務，因 提供的產品及服務之交易，亦可能會嚴重干擾本公司的
務狀況及經營結果造成負面影響。 採購集中之風險及因應措施 此可能面臨供應商供給有限且交貨期長的情況，為了優 營運。再者，隨著台積公司的製造營運擴大至部分國外區
●原物料 化供應鏈管理，台積公司針對交貨前置期進行評估及預 域，本公司就智慧財產權被侵害之風險管理亦面臨更

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是一個根據年報資料與上下文作回答的助手,"
     "如果有明確數據或技術(產品)名稱可以用數據或名稱回答,"
     "回答以繁體中文和台灣用語為主。"
     "{context}"),
    ("human","{question}")])

#simplified QA function
def question_and_answer(question):
    qa = RetrievalQA.from_llm(llm=llm_16k,
                              prompt=prompt,
                              return_source_documents=True,
                              retriever=db.as_retriever(
                                  search_kwargs={'k':10}))
    result = qa(question)
    return result

In [ ]:
while True:
    question = input("輸入問題:")
    if not question.strip():
        break
    result=question_and_answer(question)
    print(result['result'])
    print('_________')
    #print(result["source_documents"])

輸入問題:公司是否有明確成長?
根據年報中提到的資料，台積公司在民國一百一十一年的營業淨利增加了50%，稅後淨利增加了70.4%，營收增長了42.61%。這些數據顯示公司在該年度實現了明確的成長。此外，公司也提到正在擴大全球製造版圖以增加客戶信任和擴展未來的成長機會。
_________
輸入問題:公司未來一年的計畫
根據所提供的資訊，目前年報並未提供公司未來一年的具體計畫。然而，根據上下文中的一些資訊，可以看出公司未來的一些重點方向和計畫:

1. 優化製造營運：公司將致力於優化製造營運，包括數位化晶圓廠，以提高效率和生產力。

2. 增加全球產能：公司將增加台灣以外地區的產能，以擴大未來的成長潛力，接觸全球人才，並提升客戶信任。

3. 氣候變遷因應：公司將推動低碳製造，邁向淨零排放，並強化氣候韌性，評估重大氣候風險與機會對公司帶來的潛在營運與財務衝擊，並執行相關因應方案。

4. 員工發展與健康：公司將持續關注員工發展，提供培訓課程、健康檢查等福利，並鼓勵員工參與運動活動，建立運動習慣。

需要注意的是，以上僅為根據提供的片段資訊所推斷的一些可能性，具體的未來計畫還需要參考完整的年報資訊。
_________
輸入問題:


In [ ]:
from langchain.chains.summarize import load_summarize_chain

In [ ]:
key_word = ['有關市場策略的調整或變化有何提及？',
          '公司對未來一年的展望是什麼？',
          '公司的總收入是否增長，淨利潤的正負情況是否有變化？',
          '國際競爭及海外市場情況',
          '目前的研發狀況?']
data_list = []

#Compiling documents for reference data
for word in key_word:
    data = db.max_marginal_relevance_search(word)
    data_list += data

prompt_template = [("system","你的任務是生成年報摘要。"
                "我們提供年報{text}請你負責生成,"
                "且要保留重點如營收漲跌、開發項目等,"
                "最後請使用繁體中文輸出報告")]
prompt = ChatPromptTemplate.from_messages(messages=prompt_template)

In [ ]:
chain_refine_16k = load_summarize_chain(llm=llm_16k,
                                        chain_type='stuff',
                                        prompt=prompt)
print(chain_refine_16k.run(data_list))

根據台積公司的年報，該公司在環境、社會、公司治理方面面臨了一些重大風險。其中包括對員工安全與健康工作環境的評估，環境管理制度的建立，以及市場風險和貿易風險的挑戰。此外，台積公司也積極致力於氣候變遷因應，推動低碳製造，並強化氣候韌性。

在財務方面，台積公司在民國一百一十一年取得了良好的營業收入成長，稅後淨利也有顯著的增加。公司在全球市場上持續推進N3高度量產，並正積極擴大台灣以外的產能，以擴大未來的成長潛力。

此外，台積公司也在開放創新平台上取得了一些重要的成就，包括推出N4X製程技術、推動智慧手機相機效能的開發，以及成功量產先進智慧手機相機感測器技術。

在社會責任方面，台積公司持續進行多項社會公益活動，包括推動女性在科技領域中的發展、推廣孝道、以及與教育部合作推廣孝道意識。此外，公司也在環保、安全與健康管理方面獲得多項認證和獎項，展現了在永續發展方面的努力和成就。

總的來說，台積公司在多個方面都取得了積極的成果，並且在面對各種風險挑戰時，都積極採取了應對措施，展現了強大的企業韌性和永續發展的決心。


In [ ]:
from langchain.chains import LLMChain
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

word_prompt=ChatPromptTemplate.from_messages(messages=[
    ("human","從{input}聯想出4個與年報分析有關的重要關鍵字,"\
     "請確保回答具有具有關聯性、多樣性和變化性。 \n "
     "僅回覆關鍵字, 並以半形逗號與空格來分隔。不要加入其他內容"
    "")]
)
word_chain = LLMChain(llm=llm_16k, prompt=word_prompt)
output_parser.parse(word_chain('公司的營收狀況如何？')['text'])

['成長', '利潤', '支出', '現金流量']

In [ ]:
data_prompt=ChatPromptTemplate.from_messages(messages=[
    ("system","你現在是一位專業的股票分析師,"
    "你會以詳細、嚴謹的角度進行年報分析, 針對{output}作分析並提及重要數字\
    ,然後生成一份專業的趨勢分析報告。"),
    ("human","{text}")])
data_chain = LLMChain(llm=llm_16k, prompt=data_prompt)

In [ ]:
def analyze_chain(input):
    #Search documents with maximum new information
    data = db.max_marginal_relevance_search(input)

    #Create keywords for search
    word = word_chain(input)
    word_list = output_parser.parse(word['text'])

    for i in word_list:
      data += db.max_marginal_relevance_search(i,k=2)
    word_list.append(input)

    result = data_chain({'output':word_list,'text':data})

    return result['text']

In [ ]:
input = '公司的營收狀況如何？'
print(analyze_chain(input))

根據台積電的年報資訊，我們可以進行以下方面的分析:

1. 成長：台積電在營業收入淨額增加方面表現出色，主要是由於本年度平均銷售單價提高、晶圓出貨量增加及受匯率變動增加所致。另外，營業費用的增加主要是由於研究發展費用增加所致。

2. 利潤：營業毛利增加主要是平均銷售單價提高、有利的匯率變動及持續成本改善所致，部分受產能利用率下降所抵銷。營業淨利增加主要是因營業毛利增加所致。

3. 現金流：台積電在現金流方面表現出色，全年來自營業活動的現金流入約15,594億，主要包含稅後淨利及折舊、攤銷費用。投資活動的現金流出約9,440億，主要用於資本支出。

4. 市場份額：根據年報資訊，台積電持續精進互補式金氧半導體影像感測器技術，並且正在開發N6射頻技術，以支援車用電子應用，這將有助於增強在市場上的競爭力。

5. 公司的營收狀況：台積電的營收狀況表現出穩健的成長趨勢，主要是由於產品技術的持續進步和市場需求的增加。

總體而言，台積電在成長、利潤和現金流方面呈現穩健的發展趨勢，並且在技術創新和市場份額方面持續投入和努力。這些數字顯示了公司的業務穩健性和成長潛力。
